In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Görüntü Sıkıştırma

Bu görevde **görüntüleri** 🖼️ veri olarak kullanacağız.

Görüntü sıkıştırma için **K-means** kullanarak, bir görüntüdeki renkleri yalnızca en sık kullanılanlara indirgeyeceksiniz.

## 1) Python ile görüntüler

Bu bölüm, Python'da görüntüleri nasıl işlediğimizi anlamanıza yardımcı olacaktır.  

**1. bölüm için kodlama gerekmez, dikkatlice okuyun ve verilen örnekleri uygulayın.**

### 1.1) Siyah beyaz görüntüler

Bir görüntü, her bir öğesinin bir piksel olduğu bir matris olarak tanımlanabilir.  

Renk skalası 0 (siyah) ile 255 (beyaz) arasındadır.  

Böylece, matristeki her bir öğenin değeri 0 ile 255 arasında olacaktır.

<img src="https://miro.medium.com/max/1386/1*bV7S0zACdidh11ikjYpLpQ.png">

Biz “matris” dedik, siz “numpy” düşündünüz, doğru 🙂  

👇 Aşağıdaki hücreyi çalıştırarak, yalnızca “numpy” ile oluşturulmuş siyah beyaz bir görüntüyü görebilirsiniz.

In [ ]:
# 0 ile 255 arasında bir değere sahip 100 elemanlı bir vektör oluşturalım.
image_bw = np.random.uniform(low=0, high=255, size=100)

# Kare matris şeklinde yeniden şekillendirin, 10x10 piksel boyutunda bir görüntü elde ederiz.
image_bw = image_bw.reshape((10, 10))

# Son olarak oluşturulan görüntüyü göster
plt.imshow(image_bw, cmap='gray');

### 1.2) Renkli görüntüler :

Her renk, ana renklerin doğrusal bir kombinasyonudur: Kırmızı, Yeşil, Mavi  

Renkli bir görüntü, her bir ana renk için birer tane olmak üzere üç katmanın üst üste bindirilmesinden oluşur.  

Dolayısıyla her piksel için üç değerimiz vardır (Kırmızı, Yeşil, Mavi).

<img src="https://miro.medium.com/max/840/1*y0nV4cWk3KbzXSe9dIWL5g.jpeg">

👇  Yine, yalnızca `numpy` ile oluşturulmuş renkli bir görüntü görmek için aşağıdaki hücreleri çalıştırın.

In [ ]:
# 0 ile 1 arasında bir değere sahip 100*3 elemanlı bir vektör oluşturalım.
image_c = np.random.uniform(low=0., high=1., size=100*3)

# 3 renkli, 10x10 piksel boyutunda kare şeklinde bir görüntüye dönüştürün.
image_c = image_c.reshape((10, 10, 3))

# Son olarak oluşturulan görüntüyü göster
plt.imshow(image_c);

In [ ]:
# Sezgisel bir fikir edinmek için, her renk katmanını çizelim.
fig, axs = plt.subplots(1, 3, figsize=(8, 6))
colors = {0:'Reds', 1:'Greens', 2:'Blues'}

for i in colors:
    axs[i].imshow(image_c[:, :, i], cmap=colors[i])
    axs[i].set_title(colors[i])

## 2) ML için görüntüler

### 2.1) Kurulum

`skimage` kütüphanesini yükleyin, bu bize klasik görüntülere erişim sağlayacaktır.

In [ ]:
! pip install --quiet scikit-image

In [ ]:
from skimage import data
img = data.astronaut()
plt.imshow(img);

❓ What is the shape of this image?  
Assign it to `img_shape`

In [ ]:
# SENİN KODUN BURAYA

### 2.2) Renk sayısı

Görüntümüzü sıkıştırmaya veya genel olarak makine öğrenimine hazır hale getirmek için, onu **gözlemler** ve **özellikler** olarak temsil etmemiz gerekir.  

Her **piksel** bir **gözlemdir**  
Her **renk değeri (Kırmızı, Yeşil, Mavi)** bir **özelliktir**  

Görüntümüzde **262.144 gözlem** (512 * 512 piksel) ve **3 özellik** (RGB) bulunmaktadır.

❓ Görüntüyü yeniden şekillendirin:  
- Mevcut 3 boyutlu şekli olan `width x height x 3`
- `n_piksel x 3` şeklindeki 2 boyutlu matrise, burada  `n_pixels = width * height`  

Yeniden şekillendirilen görüntüyü `X`'e atayın.

In [ ]:
# SENİN KODUN BURAYA

Renkli bir görüntü, 16 milyona kadar potansiyel renk içerebilir.  

Aslında, her piksel için **3** kanalımız vardır ve her kanal için **256** olası değerimiz vardır (0'dan 255'e kadar).  
Bu nedenle, renkli bir görüntüde potansiyel olarak maksimum **16.777.216** *($256^3$)* renk vardır.  

Görüntümüz sadece 262.144 piksel (512 * 512) olduğundan, her piksel üç kanalın değerleri tarafından tanımlanan tek bir renk içerdiğinden, en fazla 262.144 renk kullanır.

Hadi sayalım! 🕵️

❓ Bu resimdeki benzersiz renklerin sayısını `color_count` içinde saklayın.

In [ ]:
# SENİN KODUN BURAYA

Elimizdekileri inceleyelim:
- 262.144 gözlem, her gözlem bir pikseldir
- Her gözlem için 3 özellik (Kırmızı, Yeşil ve Mavi değerleri)
- Bilinmeyen sayıda küme 😱

**Renk sayısını azaltmak için neler yapabileceğimize bakalım** 🎨

### 🧪 Kodunu Test Et

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('image_analysis',
                         img_shape = img_shape,
                         color_count = color_count)
result.write()
print(result.check())

## 3) K-means ile sıkıştırma

**113.382** rengi **K** renge indirmek istiyoruz.  

Pikseller üzerinde `KMeans` algoritması kullanarak, her pikseli **K** kümeden birine atayabiliriz!  

Her kümenin merkezi, o kümeye ait piksellerin ortalama rengi olacaktır. 

Daha sonra bu “ortalama küme rengini” kümedeki her piksel için RGB değeri olarak kullanabiliriz.  

💪🧠 Haydi başlayalım, hedefimiz sadece **32** renk kullanmak!

❓ ML hazır görüntünüz `X` üzerinde `n_clusters=32` ile bir K-means uyarlayın ve bunu `kmeans`'e atayın.

In [ ]:
# SENİN KODUN BURAYA

👉 `kmeans`'inizin `labels_`'ini kontrol edin, ardından `shape` ve `number of unique values`'ını kontrol edin.

In [ ]:
# SENİN KODUN BURAYA

☝️ Şöyle açıklayalım:

- Her etiket bir kümedir
- Her gözleme bir etiket atanmıştır
- Toplamda 32 farklı etiket vardır, her küme için bir tane

❓ `KMeans`'inizin `cluster_centers_` öğesini, şeklini ve ilk öğesini kontrol edin.

In [ ]:
# SENİN KODUN BURAYA

☝️Her cluster_center, RGB değerlerinin bir vektörüdür ve kümenin ortalama rengini temsil eder.

❓ Her piksel resmi için karşılık gelen kümenin ortalama rengini depolayan `X_compressed` dizisini oluşturun.

<details span="markdown">
    <summary>💡 Yardım</summary>

Bunu yalnızca `kmeans.cluster_centers_` ve `kmeans.labels_` kullanarak gerçekleştirebilirsiniz.

---

</details>

In [ ]:
# SENİN KODUN BURAYA

RGB değerleri tamsayı olmalıdır.  

`X_compressed` içindeki renklerimiz `float64` türündedir.  

❓ `X_compressed` türünü, 0 ile 255 arasında değişen, işaretsiz 8 bitlik tamsayı türü olan `uint8` türüne dönüştürün.

In [ ]:
# SENİN KODUN BURAYA

❓ Kullanılan benzersiz renk sayısının gerçekten 32 olduğunu doğrulayın.

In [ ]:
# SENİN KODUN BURAYA

Neredeyse bitti! 💪

## 4) Sıkıştırılmış görüntüyü çiz

`X_compressed` şeklimiz (262144, 3) olduğundan, bunu bir görüntü olarak görüntüleyemeyiz.

❓ Sıkıştırılmış görüntünüzü yeniden şekillendirin ve orijinal görüntünüzle yan yana çizim yapın.  

<details>
    <summary>💡 İpucu</summary>

Düzleştirilmiş `X_compressed` öğenizi bir görüntü için doğru boyutlara yeniden şekillendirmeniz gerekir.  
Orijinal görüntünüz bu boyutlara sahiptir.

---

</details>

In [ ]:
# SENİN KODUN BURAYA

Aferin! 💪

Bazı renkler kaybolmuş, ancak orijinal görüntüyü kolayca tanıyabilirsiniz.

🏁 **Not defterinizi itmeyi unutmayın.**  

Günün görevlerini yerine getirin ve zamanınız olursa buraya geri dönün 😉

## 5 - İsteğe bağlı) elbow yöntemini kullanın

En az renk bilgisini kaybeden optimum sıkıştırmayı bulmak için elbow yöntemini kullanmayı deneyebilirsiniz.   

Örneğin, [5, 10, 20, 30, 50, 70, 100] listesindeki `n_clusters` için `inertia` değerini çizmeye çalışın.  

⚠️ Uzun süren eğitim, birkaç dakika beklemeniz gerekebilir.

In [ ]:
# SENİN KODUN BURAYA

❓ Aşağıdaki hücreyi çalıştırarak, farklı sayıda küme kullanarak sıkıştırılmış görüntünüzü görselleştirin  
⚠️ Bu uzun bir eğitimdir, birkaç dakika beklemeniz gerekecektir

In [ ]:
wcss = []
compressed_astronaut = []
for i in [1, 2, 5, 10, 20, 30, 50, 70, 100]:
    print('working with ' + str(i) + ' clusters...', flush=True)
    kmeans = KMeans(n_clusters = i, max_iter=10)
    kmeans.fit(X)
    # inertia hesapla
    wcss.append(kmeans.inertia_)
    # sıkıştırılmış astronot hesapla
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    X_compressed = centers[labels]
    X_compressed = X_compressed.astype('uint8')
    img_compressed = X_compressed.reshape(img_shape[0], img_shape[1], img_shape[2])
    compressed_astronaut.append(img_compressed)
    # print edin
    print("-"*120)
    print(f"If we compress our 113382 unique colors into {i} clusters...")
    print(f"The Kmeans inertia is equal to {kmeans.inertia_}")
    fig, ax = plt.subplots(1, 2, figsize = (7, 7))
    ax[0].imshow(img)
    ax[0].set_title('Original Image')
    ax[1].imshow(img_compressed)
    ax[1].set_title('Compressed Image')
    for ax in fig.axes:
        ax.axis('off')
    plt.tight_layout()
    plt.show()